In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.5 MB/s eta 0:00:00


In [2]:
import optuna
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [3]:
data = load_iris()
X, y = data.data, data.target
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42) # necesitamos un conjunto de validación para obtener hiperparámetros.

In [4]:
import pandas as pd
y_train = pd.get_dummies(y_train).values # son 3 clases!
y_valid = pd.get_dummies(y_valid).values

In [5]:
def objective(trial):

    num_layers = trial.suggest_int('num_layers', 1, 3)
    model = Sequential()

    for i in range(num_layers):
        num_units = trial.suggest_int(f'n_units_layer_{i}', 4, 128) # la cantidad de neuronas de cada capa tambien se puede pasar como hiperparámetro
        # activations = trial.suggest_categorical(f'')
        model.add(Dense(num_units, activation='relu')) # capas densas con activacion ReLU

    # capa de salida
    model.add(Dense(3, activation='softmax')) # 3 son las clases de salida

    # compilar
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # entrenar
    model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=5, batch_size=32, verbose=0)

    # evaluar
    score = model.evaluate(X_valid, y_valid, verbose=0)
    return score[1]

# crear un estudio de Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# obtener los mejores hiperparámetros
best_params = study.best_params
print("Best parámetros encontrados:", best_params)


[I 2024-05-23 17:53:08,402] A new study created in memory with name: no-name-14d01ec6-4017-4d20-9272-7a175a2f05d1
[I 2024-05-23 17:53:09,924] Trial 0 finished with value: 0.30000001192092896 and parameters: {'num_layers': 2, 'n_units_layer_0': 121, 'n_units_layer_1': 6}. Best is trial 0 with value: 0.30000001192092896.
[I 2024-05-23 17:53:11,135] Trial 1 finished with value: 0.6333333253860474 and parameters: {'num_layers': 1, 'n_units_layer_0': 80}. Best is trial 1 with value: 0.6333333253860474.
[I 2024-05-23 17:53:12,654] Trial 2 finished with value: 0.8333333134651184 and parameters: {'num_layers': 2, 'n_units_layer_0': 83, 'n_units_layer_1': 48}. Best is trial 2 with value: 0.8333333134651184.
[I 2024-05-23 17:53:14,897] Trial 3 finished with value: 0.36666667461395264 and parameters: {'num_layers': 1, 'n_units_layer_0': 49}. Best is trial 2 with value: 0.8333333134651184.
[I 2024-05-23 17:53:16,578] Trial 4 finished with value: 0.699999988079071 and parameters: {'num_layers': 3, 

Best parámetros encontrados: {'num_layers': 3, 'n_units_layer_0': 99, 'n_units_layer_1': 86, 'n_units_layer_2': 15}
